In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --force-reinstall

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import numpy as np
print("NumPy version:", np.__version__)
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

NumPy version: 1.26.4
TensorFlow version: 2.20.0-dev20250425


In [5]:
import warnings
warnings.filterwarnings( 'ignore' )

import gc, os, time, numpy as np, pandas as pd, datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
import gc, os, time,datetime, numpy as np, pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split

import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import text, sequence

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import Input, Reshape, Embedding, BatchNormalization
from tensorflow.keras.layers import GRU, LSTM, Conv1D, Conv2D, Dense, Bidirectional
from tensorflow.keras.layers import SpatialDropout1D, Dropout, Concatenate, concatenate
from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling1D, AveragePooling1D

from contextlib import redirect_stdout
from tensorflow.keras.utils import plot_model
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # supress tensorflow warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

PRE_TRAINED_FLAG = True
EMBEDDING_MATRIX = []

result_path =  '/content/drive/MyDrive/T2-Code/T2-DL-FastText-off-Result.csv'
file_path =  '/content/drive/MyDrive/T2-Code/Offensive-24K-T2.xlsx'
EMBEDDING_FILE = '/content/drive/MyDrive/T2-Code/Embedding/cc.ur.300.off.vec'

result_path =  r'C:\Users\mojua\Desktop\DL-Code\T2-DL-FastText-off-Result.csv'
file_path = r'C:\Users\mojua\Desktop\DL-Code\Dataset\Offensive-24K-T2.xlsx'
EMBEDDING_FILE = r'C:\Users\mojua\Desktop\DL-Code\T1-Code\Embeddings\cc.ur.300.off.vec'
########################################################
def WriteResutls(reports):

  unt0 = {'precision':[], 'recall':[], 'f1-score':[] }
  tin1 = {'precision':[], 'recall':[], 'f1-score':[] }
  macroavg = {'precision':[], 'recall':[], 'f1-score':[] }
  weightedavg = {'precision':[], 'recall':[], 'f1-score':[] }
  accu = []
  for report in reports:
    for k,v in report.items():
      if 'UNT' in k:
        unt0['precision'].append(v['precision'])
        unt0['recall'].append(v['recall'])
        unt0['f1-score'].append(v['f1-score'])

      elif 'TIN' in k:
        tin1['precision'].append(v['precision'])
        tin1['recall'].append(v['recall'])
        tin1['f1-score'].append(v['f1-score'])

      elif 'macro avg' in k:
        macroavg['precision'].append(v['precision'])
        macroavg['recall'].append(v['recall'])
        macroavg['f1-score'].append(v['f1-score'])

      elif 'weighted avg' in k:
        weightedavg['precision'].append(v['precision'])
        weightedavg['recall'].append(v['recall'])
        weightedavg['f1-score'].append(v['f1-score'])
      else:
        accu.append(v)

  print('Accuracy:',np.mean(accu))
  print("")
  print('UNT 0 Precision:',np.mean(unt0['precision']))
  print('UNT 0 Recall:',np.mean(unt0['recall']))
  print('UNT 0 F1-Score:',np.mean(unt0['f1-score']))
  print("")
  print('TIN 1 Precision:',np.mean(tin1['precision']))
  print('TIN 1 Recall:',np.mean(tin1['recall']))
  print('TIN 1 F1-Score:',np.mean(tin1['f1-score']))

  print("")
  print('Weighted Avg Precision:',np.mean(weightedavg['precision']))
  print('Weighted Avg Recall:',np.mean(weightedavg['recall']))
  print('Weighted Avg F1-Score:',np.mean(weightedavg['f1-score']))

  print("")
  print('Macro  Precision:',np.mean(macroavg['precision']))
  print('Macro  Recall:',np.mean(macroavg['recall']))
  print('Macro  F1-Score:',np.mean(macroavg['f1-score']))

  file = open( result_path, mode='a' )
  file.write( modelname+ ' ( '+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +') \n' )
  file.write( 'Accuracy:'+str(np.mean(accu))+'\n' )
  file.write('UNT 0 Precision:'+str(np.mean(unt0['precision']))+'\n' )
  file.write('UNT 0 Recall:'+str(np.mean(unt0['recall']))+'\n' )
  file.write('UNT 0 F1-Score:'+str(np.mean(unt0['f1-score']))+'\n' )

  file.write('TIN 1 Precision:'+str(np.mean(tin1['precision']))+'\n' )
  file.write('TIN 1 Recall:'+str(np.mean(tin1['recall']))+'\n' )
  file.write('TIN 1 F1-Score:'+str(np.mean(tin1['f1-score']))+'\n' )

  file.write('Weighted Avg Precision:'+str(np.mean(weightedavg['precision']))+'\n' )
  file.write('Weighted Avg Recall:'+str(np.mean(weightedavg['recall']))+'\n' )
  file.write('Weighted Avg F1-Score:'+str(np.mean(weightedavg['f1-score']))+'\n' )

  file.write('Macro  Precision:'+str(np.mean(macroavg['precision']))+'\n' )
  file.write('Macro  Recall:'+str(np.mean(macroavg['recall']))+'\n' )
  file.write('Macro  F1-Score:'+str(np.mean(macroavg['f1-score']))+'\n' )
  file.close()
  print("Done")


######################################################################################
def hms_string( sec_elapsed ):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{} hrs {:>02} mins {:>05.2f} secs".format( h, m, s )
########################################################################################
def get_coefs( word, *arr ):
    return word, np.asarray( arr, dtype='float32' )

def get_vectors( tokenizer, emb_mean, emb_std, embed_size ):
    hits = 0
    misses = 0
    word_index = tokenizer.word_index
    num_words = len( word_index ) + 1
    embedding_matrix = np.random.normal( emb_mean, emb_std, ( num_words, embed_size ) ) # for unk tokens
    embedding_matrix[0] = np.zeros( ( embed_size ) ) # replace zero padding with zeros
    for word, i in word_index.items(  ):
        embedding_vector = EMBEDDINGS_INDEX.get( word )
        if embedding_vector is not None:
            embedding_matrix[ i ] = embedding_vector
            hits += 1
        else:
            misses += 1
    print( "Pretrained Embeddings: %d hits (%d misses)" % ( hits, misses ) )
    gc.collect()
    return embedding_matrix

#################################################################################################################

df = pd.read_excel(file_path, engine='openpyxl')
df.reset_index( drop=True, inplace=True )
df.Tweet = df.Tweet.astype( 'str' )
df.dropna( inplace=True )
df.reset_index( drop=True, inplace=True )
print(df.head())
print(df.info())
print(df.columns, df.shape)
gc.collect()
xcolumn = 'Tweet'
ycolumn = 'Tag'


   Unnamed: 0                                              Tweet  Tag
0           0  USER دو بے نسلئیے ، حرامخور منافق مل رہے ہیں پ...    0
1           3                   USER گھٹیا انسان دنیا ہی چھوڑ دو    1
2          11  USER PMLN میں آپ کے بارے میں میری بہتر راۓ تھی...    1
3          15  USER اسپین کی ٹیم بھی پاکستان کی کرکٹ ٹیم کی ط...    0
4          20  ہمیں تو آج تک سمجھ نہیں آئی کہ کم عقل عیسائی ح...    1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8758 non-null   int64 
 1   Tweet       8758 non-null   object
 2   Tag         8758 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 205.4+ KB
None
Index(['Unnamed: 0', 'Tweet', 'Tag'], dtype='object') (8758, 3)


In [7]:
def optimize_threshold( y_true, y_pred ):
    thresholds = np.arange( 0.1, 0.9, 0.001 )
    vscores = np.zeros( thresholds.shape[ 0 ] )
    for th in range( thresholds.shape[ 0 ] ):
        vscores[ th ] = f1_score( y_true, ( y_pred >= thresholds[ th ] ).astype( 'int32' ) )
    return thresholds[ np.argmax( vscores ) ] # return threshold that has max val score

class F1_score_callback( Callback ):
    def __init__( self, val_data, filepath, patience=10, decay=True, decay_rate=0.1, decay_after=2 ):
        self.file_path = filepath
        self.patience = patience
        self.patience_counter = 0
        self.decay = decay
        self.decay_rate = decay_rate
        self.decay_after = decay_after
        self.validation_data = val_data

    def __optimize_threshold_for_f1( self, y_true, y_pred ):
        thresholds = np.arange( 0.1, 0.9, 0.001 )
        vscores = np.zeros( thresholds.shape[ 0 ] )
        for th in range( thresholds.shape[ 0 ] ):
            vscores[ th ] = f1_score( y_true, ( y_pred >= thresholds[ th ] ).astype( 'int32' ) )
        return thresholds[ np.argmax( vscores ) ]

    def on_train_begin( self, logs=None ):
        self.val_f1s = []
        self.best_val_f1 = np.NINF

    def on_epoch_end( self, epoch, logs={} ):
        val_predict = self.model.predict( self.validation_data[ 0 ] )
        val_targ = self.validation_data[ 1 ]

        threshold = self.__optimize_threshold_for_f1( val_targ, val_predict )

        _val_f1 = f1_score( val_targ, ( val_predict >= threshold ).astype( 'int32' ) )
        self.val_f1s.append(_val_f1)

        self.patience_counter = self.patience_counter + 1

        printstatement = 'Epoch: {:03d}'.format( epoch + 1 ) +\
        ' --LR: {:1.0e}'.format( K.get_value( self.model.optimizer.learning_rate ) ) +\
        ' --MaxValF1: {:0.7f}'.format( max( self.val_f1s ) ) +\
        ' --CurValF1: {:0.7f}'.format( _val_f1 ) + ' --Patience: {:02d}'.format( self.patience_counter )

        if _val_f1 > self.best_val_f1:
            self.model.save( self.file_path, overwrite=True, include_optimizer=True )
            self.best_val_f1 = _val_f1
            printstatement = printstatement + ' --F1 improved: {:0.7f}'.format( self.best_val_f1 )
            self.patience_counter = 0

        print( printstatement )

        if ( (self.decay==True) & ((self.patience_counter % self.decay_after) == 0) & (self.patience_counter != 0) ):
          if isinstance(self.model.optimizer.learning_rate, tf.Variable):
            K.set_value(self.model.optimizer.learning_rate, (K.get_value( self.model.optimizer.learning_rate ) * self.decay_rate))
          else:
            self.model.optimizer.learning_rate = K.get_value( self.model.optimizer.learning_rate ) * self.decay_rate  # just assign directly

            #K.set_value( self.model.optimizer.learning_rate, ( K.get_value( self.model.optimizer.learning_rate ) * self.decay_rate ) )

        if self.patience_counter == self.patience:
            self.model.stop_training = True
            print( 'Training stopped due to the patience parameter.' + \
                  ' --Patience: {:02d}'.format( self.patience_counter ) )

        return

def focal_loss( gamma=1., alpha=.1 ):
    def focal_loss_fixed( y_true, y_pred ):
        pt_1 = tf.where( tf.equal( y_true, 1 ), y_pred, tf.ones_like( y_pred ) )
        pt_0 = tf.where( tf.equal( y_true, 0 ), y_pred, tf.zeros_like( y_pred ) )
        return -K.sum( alpha * K.pow( 1. - pt_1, gamma ) * K.log( pt_1 ) ) - K.sum( ( 1 - alpha ) * K.pow( pt_0, gamma ) * K.log( 1. - pt_0 ) )
    return focal_loss_fixed

def MCBiGRU(
    max_len=100, embed_inp=10000, embed_size=300, embedding_matrix=[], embed_trainable=False,
    drpt=.1, emb_weights_init='', conv_weights_init='', conv_act='', fc_weights_init='',
    fc_act='', optimizer='' ):

    window_sizes = [ 1,2,3,5,6 ]
    allchannels = [  ]

    inp = Input( shape=( max_len, ) )
    if embedding_matrix is None:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      embeddings_initializer=emb_weights_init, trainable=embed_trainable )( inp )
    else:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      weights=[ embedding_matrix ], trainable=embed_trainable )( inp )

    for j,i in enumerate( window_sizes ):
        conv = Conv1D( filters=128, kernel_size=i, padding='valid',
                      activation=conv_act, kernel_initializer=conv_weights_init )( x )
        drpt1 = Dropout( drpt )( conv )
        maxpool = MaxPooling1D( pool_size=4 )( drpt1 )
        bgru = Bidirectional( GRU( 200, return_sequences=False ) )( maxpool )
        drpt2 = Dropout( drpt )( bgru )
        allchannels.append( drpt2 )

    z = concatenate( allchannels )
    fc = Dense( 2000, activation=fc_act, kernel_initializer=fc_weights_init )( z )
    bnorm = BatchNormalization(  )( fc )
    output = Dense( 1, activation='sigmoid' )( bnorm )

    model = Model( inputs=inp, outputs=output )
    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model
############################################################################
def CNN_George(
    max_len=100, embed_inp=10000, embed_size=300, embedding_matrix=[], embed_trainable=False,
    emb_weights_init='', optimizer='' ):

    filter_sizes = [ 3,4,5 ]
    num_filters = 32
    inp = Input( shape=( max_len, ) )
    if embedding_matrix is None:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      embeddings_initializer=emb_weights_init, trainable=embed_trainable )( inp )
    else:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      weights=[ embedding_matrix ], trainable=embed_trainable )( inp )

    x = Reshape( ( max_len, embed_size, 1 ) )( x )
    pooled = [  ]
    for j,i in enumerate( filter_sizes ):
        conv = Conv2D( num_filters, kernel_size=( i, embed_size ), activation='relu' )( x )
        globalmax = GlobalMaxPooling2D(  )( conv )
        pooled.append( globalmax )
    z = Concatenate( axis=1 )( pooled )
    output = Dense( 1, activation='sigmoid' )( z )

    model = Model( inputs=inp, outputs=output )

    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model
##################################################################################################

In [9]:
modelname = 'CNN_George-ft'
modelpath = './Saved Models/' + modelname + '/'
modelresults = './Model Results'
modelsummaries = './Model - Summaries-Figures'
DIRECTORIES_TO_BE_CREATED = [ modelpath, modelresults, modelsummaries ]
for directory in DIRECTORIES_TO_BE_CREATED:
    if not os.path.exists( directory ): os.makedirs( directory )

# hyper parameters for this model
MAX_LEN = 128
EMBED_SIZE = 300
EMBED_TRAINABLE = False
EMB_WEIGHTS_INIT = 'he_normal'
LR_RATE = 1e-3
OPTIMIZER = 'adam'
BATCH = 32
NEPOCHS = 30
PATIENCE = 5
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

if PRE_TRAINED_FLAG == True:
    EMBEDDINGS_INDEX = dict( get_coefs( *o.strip().split() ) for o in open( EMBEDDING_FILE, encoding='utf-8' )  if len(o)>19 )
    print( 'Total Words in Pretrained Embeddings: ', len( EMBEDDINGS_INDEX.values() ) )
    all_embs = np.stack( list(EMBEDDINGS_INDEX.values()) )
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    print( 'Pretrained Embeddings mean:', emb_mean, ' std: ', emb_std )

Total Words in Pretrained Embeddings:  1154697
Pretrained Embeddings mean: -0.0023497287  std:  0.08804372


In [13]:
warnings.filterwarnings( 'ignore' )
import datetime
from tensorflow.keras.layers import Input, Reshape, Embedding, BatchNormalization
start_time = time.time()
print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
com_text, com_label, com_predicted, com_prob = [], [], [], []
com_indices = []
hist = {}
reports = []

skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
print(skf)

fold = 1
for train_index, test_index in skf.split( df[ xcolumn ], df[ ycolumn ] ):
    # seprating train and test sets
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    xtrain = df.loc[ train_index ][ xcolumn ].values
    xtest = df.loc[ test_index ][ xcolumn ].values

    ytrain = df.loc[ train_index ][ ycolumn ].values
    ytest = df.loc[ test_index ][ ycolumn ].values

    # splitting train and validation sets
    xtrain, xval, ytrain, yval = train_test_split( xtrain, ytrain, test_size=0.15, random_state=0 )

    # tokenization with keras tokenizer
    tokenizer = text.Tokenizer(  )
    tokenizer.fit_on_texts( np.append( xtrain, xval ) )

    xtrain = tokenizer.texts_to_sequences( xtrain )
    xval = tokenizer.texts_to_sequences( xval )
    xtest = tokenizer.texts_to_sequences( xtest )

    # pad the tokenized sequences
    xtrain = sequence.pad_sequences( xtrain, maxlen=MAX_LEN )
    xval = sequence.pad_sequences( xval, maxlen=MAX_LEN )
    xtest = sequence.pad_sequences( xtest, maxlen=MAX_LEN )

    # check if pre-trained word embeddings flag is true
    if PRE_TRAINED_FLAG == True:
        EMBEDDING_MATRIX = get_vectors(
            tokenizer=tokenizer, emb_mean=emb_mean, emb_std=emb_std, embed_size=EMBED_SIZE
        )

    # define a model
    EMBED_INP_SIZE = len( tokenizer.word_index ) + 1
    model = CNN_George(
        max_len=MAX_LEN, embed_inp=EMBED_INP_SIZE, embed_size=EMBED_SIZE,
        embedding_matrix=EMBEDDING_MATRIX, embed_trainable=EMBED_TRAINABLE,
        emb_weights_init=EMB_WEIGHTS_INIT, optimizer='adam'
    )

    # save model summaries and model architecture diagrams in the first fold only
    if fold == 1:
       # plot_model( model=model, to_file='{}/{}.png'.format( modelsummaries, modelname ), show_shapes=False )

        with open( '{}/{}.txt'.format( modelsummaries, modelname ), 'w' ) as s:
            with redirect_stdout( s ):model.summary()
    # Change the type of LR_RATE to float
    #LR_RATE = float(LR_RATE)
    #K.set_value( model.optimizer.learning_rate, LR_RATE )
    LR_RATE = float(LR_RATE)
    if isinstance(model.optimizer.learning_rate, tf.Variable):
      K.set_value(model.optimizer.learning_rate, LR_RATE)
    else:
      model.optimizer.learning_rate = LR_RATE  # just assign directly
    # train the model with callbacks for early stopping
    f1callback = F1_score_callback( val_data=( xval, yval ), filepath=modelpath + modelname + str( fold ) + ".keras",
        patience=PATIENCE, decay=DECAY, decay_rate=DECAY_RATE, decay_after=DECAY_AFTER )
    histmodel = model.fit(xtrain, ytrain, batch_size=BATCH, epochs=NEPOCHS, verbose=0, callbacks=[ f1callback ] )
    # save history of all folds
    hist[ 'fold' + str( fold ) ] = histmodel.history.copy()
    # delete trained model object
    del model, histmodel, f1callback
    EMBEDDING_MATRIX = []
    K.clear_session()
    gc.collect()
    # delete pre trained embeddings from the memory
    if ( PRE_TRAINED_FLAG == True ) and ( fold == skf.n_splits ):
        del EMBEDDINGS_INDEX
        gc.collect()

    # load saved model
    loaded_model = load_model( modelpath + modelname + str( fold ) + ".keras")

    # get predictions (probabilities) for validation and test sets respectively
    valpredictions = loaded_model.predict( xval, verbose=0, batch_size=BATCH )
    testpredictions = loaded_model.predict( xtest, verbose=0, batch_size=BATCH )

    # delete loaded model
    del loaded_model
    K.clear_session()
    gc.collect()

    # optimizer threshold on validation set
    threshold = optimize_threshold( yval, valpredictions )

    # save accuracy, precision, recall, f1 and confusion matrices
    vallabels = ( valpredictions >= threshold ).astype( 'int32' )
    testlabels = ( testpredictions >= threshold ).astype( 'int32' )

    testaccuracy.append( accuracy_score( ytest, testlabels ) )
    testprecision.append( precision_score( ytest, testlabels ) )
    testrecall.append( recall_score( ytest, testlabels ) )
    testf1.append( f1_score( ytest, testlabels ) )
    testcm.append( confusion_matrix( ytest, testlabels ) )
    reports.append(classification_report( ytest, testlabels, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))

    print( 'Fold: {:02d} out of {:02d} completed.'.format( fold, skf.get_n_splits() ) )
    print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

    fold = fold + 1
time_took = time.time() - start_time
print( f"Total runtime: { hms_string( time_took ) }" )
WriteResutls(reports)

Local System Time: 10:34 AM
StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
2025-05-23 10:34:27


Model: "functional_1"

+----------------------------------------------------------------------------------------------------------+
| Layer (type)                  | Output Shape              |         Param # | Connected to               |
|-------------------------------+---------------------------+-----------------+----------------------------|
| input_layer_1 (InputLayer)    | (None, 128)               |               0 | -                          |
|-------------------------------+---------------------------+-----------------+----------------------------|
| embedding_1 (Embedding)       | (None, 128, 300)          |       5,611,800 | input_layer_1[0][0]        |
|-------------------------------+---------------------------+-----------------+----------------------------|
| reshape_1 (Reshape)           | (None, 128, 300, 1)       |               0 | embedding_1[0][0]          |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv2d_3 (Conv2D)             | (None, 126, 1, 32)        |          28,832 | reshape_1[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv2d_4 (Conv2D)             | (None, 125, 1, 32)        |          38,432 | reshape_1[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv2d_5 (Conv2D)             | (None, 124, 1, 32)        |          48,032 | reshape_1[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| global_max_pooling2d_3        | (None, 32)                |               0 | conv2d_3[0][0]             |
| (GlobalMaxPooling2D)          |                           |                 |                            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| global_max_pooling2d_4        | (None, 32)                |               0 | conv2d_4[0][0]             |
| (GlobalMaxPooling2D)          |                           |                 |                            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| global_max_pooling2d_5        | (None, 32)                |               0 | conv2d_5[0][0]             |
| (GlobalMaxPooling2D)          |                           |                 |                            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| concatenate_1 (Concatenate)   | (None, 96)                |               0 | global_max_pooling2d_3[0]… |
|                               |                           |                 | global_max_pooling2d_4[0]… |
|                               |                           |                 | global_max_pooling2d_5[0]… |
|-------------------------------+---------------------------+-----------------+----------------------------|
| dense_1 (Dense)               | (None, 1)                 |              97 | concatenate_1[0][0]        |
+----------------------------------------------------------------------------------------------------------+

 Total params: 5,727,193 (21.85 MB)

 Trainable params: 115,393 (450.75 KB)

 Non-trainable params: 5,611,800 (21.41 MB)

Pretrained Embeddings: 14041 hits (4664 misses)
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
Epoch: 001 --LR: 1e-03 --MaxValF1: 0.8682505 --CurValF1: 0.8682505 --Patience: 01 --F1 improved: 0.8682505
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Epoch: 002 --LR: 1e-03 --MaxValF1: 0.8687430 --CurValF1: 0.8687430 --Patience: 01 --F1 improved: 0.8687430
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Epoch: 003 --LR: 1e-03 --MaxValF1: 0.8705613 --CurValF1: 0.8705613 --Patience: 01 --F1 improved: 0.8705613
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Epoch: 004 --LR: 1e-03 --MaxValF1: 0.8712422 --CurValF1: 0.8712422 --Patience: 01 --F1 improved: 0.8712422
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Epoch: 005 --LR: 1e-03 --MaxValF1: 0.8732394 --CurValF1: 0.8732394 --Patience: 01 --F1 improved: 0.8732394
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Epoch: 006 --LR: 1e-03 --MaxValF1: 0.8732394 --CurValF1: 0.8713418 --Patience: 01
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Epoch: 007 --LR: 3e-04 --MaxValF1: 0.8742382 --CurValF1

In [15]:
modelname = 'MCBiGRU-ft'
modelpath = './Saved Models/' + modelname + '/'
modelresults = './Model Results'
modelsummaries = './Model - Summaries-Figures'
DIRECTORIES_TO_BE_CREATED = [ modelpath, modelresults, modelsummaries ]
for directory in DIRECTORIES_TO_BE_CREATED:
    if not os.path.exists( directory ): os.makedirs( directory )
# hyper parameters for this model
MAX_LEN = 128
EMBED_SIZE = 300
EMBED_TRAINABLE = False

EMB_WEIGHTS_INIT = 'he_normal'
DRPT = 0.4
CONV_WEIGHTS_INIT = 'he_uniform'
CONV_ACT = 'elu'
FC_WEIGHTS_INIT = 'he_uniform'
FC_ACT = 'elu'
LR_RATE = 1e-3
OPTIMIZER = 'adam'

BATCH = 32
NEPOCHS = 30
PATIENCE = 5
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

if PRE_TRAINED_FLAG == True:
    EMBEDDINGS_INDEX = dict( get_coefs( *o.strip().split() ) for o in open( EMBEDDING_FILE, encoding='utf-8' )  if len(o)>19 )
    print( 'Total Words in Pretrained Embeddings: ', len( EMBEDDINGS_INDEX.values() ) )
    all_embs = np.stack( list(EMBEDDINGS_INDEX.values()) )
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    print( 'Pretrained Embeddings mean:', emb_mean, ' std: ', emb_std )

Total Words in Pretrained Embeddings:  1154697
Pretrained Embeddings mean: -0.0023497287  std:  0.08804372


In [17]:
# Execute this cell
warnings.filterwarnings( 'ignore' )
start_time = time.time()
print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
com_text, com_label, com_predicted, com_prob = [], [], [], []
com_indices = []
hist = {}
reports = []

skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
print(skf)
fold = 1
for train_index, test_index in skf.split( df[ xcolumn ], df[ ycolumn ] ):
    # seprating train and test sets
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    xtrain = df.loc[ train_index ][ xcolumn ].values
    xtest = df.loc[ test_index ][ xcolumn ].values

    ytrain = df.loc[ train_index ][ ycolumn ].values
    ytest = df.loc[ test_index ][ ycolumn ].values

    # splitting train and validation sets
    xtrain, xval, ytrain, yval = train_test_split( xtrain, ytrain, test_size=0.15, random_state=0 )

    # tokenization with keras tokenizer
    tokenizer = text.Tokenizer(  )
    tokenizer.fit_on_texts( np.append( xtrain, xval ) )

    xtrain = tokenizer.texts_to_sequences( xtrain )
    xval = tokenizer.texts_to_sequences( xval )
    xtest = tokenizer.texts_to_sequences( xtest )

    # pad the tokenized sequences
    xtrain = sequence.pad_sequences( xtrain, maxlen=MAX_LEN )
    xval = sequence.pad_sequences( xval, maxlen=MAX_LEN )
    xtest = sequence.pad_sequences( xtest, maxlen=MAX_LEN )

    # check if pre-trained word embeddings flag is true
    if PRE_TRAINED_FLAG == True:
        EMBEDDING_MATRIX = get_vectors(
            tokenizer=tokenizer, emb_mean=emb_mean, emb_std=emb_std, embed_size=EMBED_SIZE
        )

    # define a model
    EMBED_INP_SIZE = len( tokenizer.word_index ) + 1
    model = MCBiGRU(max_len=MAX_LEN, embed_inp=EMBED_INP_SIZE, embed_size=EMBED_SIZE,
        embedding_matrix=EMBEDDING_MATRIX, embed_trainable=EMBED_TRAINABLE, drpt=DRPT, emb_weights_init=EMB_WEIGHTS_INIT,
        conv_weights_init=CONV_WEIGHTS_INIT, conv_act=CONV_ACT,fc_weights_init=FC_WEIGHTS_INIT, fc_act=FC_ACT, optimizer=OPTIMIZER )

    # save model summaries and model architecture diagrams in the first fold only
    if fold == 1:
        plot_model( model=model, to_file='{}/{}.png'.format( modelsummaries, modelname ), show_shapes=False )

        with open( '{}/{}.txt'.format( modelsummaries, modelname ), 'w' ) as s:
            with redirect_stdout( s ):model.summary()

    if isinstance(model.optimizer.learning_rate, tf.Variable):
      K.set_value(model.optimizer.learning_rate, LR_RATE)
    else:
      model.optimizer.learning_rate = LR_RATE
    filepath = modelpath + modelname + str( fold ) + ".h5"

    # train the model with callbacks for early stopping
    f1callback = F1_score_callback(val_data=( xval, yval ), filepath=filepath,
        patience=PATIENCE, decay=DECAY, decay_rate=DECAY_RATE, decay_after=DECAY_AFTER)
    histmodel = model.fit(xtrain, ytrain, batch_size=BATCH, epochs=NEPOCHS, verbose=0, callbacks=[ f1callback ] )

    # save history of all folds
    hist[ 'fold' + str( fold ) ] = histmodel.history.copy()

    # delete trained model object
    del model, histmodel, f1callback
    EMBEDDING_MATRIX = []
    K.clear_session()
    gc.collect()
    # delete pre trained embeddings from the memory
    if ( PRE_TRAINED_FLAG == True ) and ( fold == skf.n_splits ):
        del EMBEDDINGS_INDEX
        gc.collect()

    # load saved model
    loaded_model = load_model( filepath )

    # get predictions (probabilities) for validation and test sets respectively
    valpredictions = loaded_model.predict( xval, verbose=0, batch_size=BATCH )
    testpredictions = loaded_model.predict( xtest, verbose=0, batch_size=BATCH )

    # delete loaded model
    del loaded_model
    K.clear_session()
    gc.collect()

    # optimizer threshold on validation set
    threshold = optimize_threshold( yval, valpredictions )

    # save accuracy, precision, recall, f1 and confusion matrices
    vallabels = ( valpredictions >= threshold ).astype( 'int32' )
    testlabels = ( testpredictions >= threshold ).astype( 'int32' )

    valaccuracy.append( accuracy_score( yval, vallabels ) )
    valprecision.append( precision_score( yval, vallabels ) )
    valrecall.append( recall_score( yval, vallabels ) )
    valf1.append( f1_score( yval, vallabels ) )
    valcm.append( confusion_matrix( yval, vallabels ) )

    testaccuracy.append( accuracy_score( ytest, testlabels ) )
    testprecision.append( precision_score( ytest, testlabels ) )
    testrecall.append( recall_score( ytest, testlabels ) )
    testf1.append( f1_score( ytest, testlabels ) )
    testcm.append( confusion_matrix( ytest, testlabels ) )
    reports.append(classification_report( ytest, testlabels, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))


    print( 'Fold: {:02d} out of {:02d} completed.'.format( fold, skf.get_n_splits() ) )
    print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

    fold = fold + 1
time_took = time.time() - start_time
print( f"Total runtime: { hms_string( time_took ) }" )
WriteResutls(reports)

Local System Time: 12:13 PM
StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
2025-05-23 12:13:26
Pretrained Embeddings: 14041 hits (4664 misses)
You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


Model: "functional"

+----------------------------------------------------------------------------------------------------------+
| Layer (type)                  | Output Shape              |         Param # | Connected to               |
|-------------------------------+---------------------------+-----------------+----------------------------|
| input_layer (InputLayer)      | (None, 128)               |               0 | -                          |
|-------------------------------+---------------------------+-----------------+----------------------------|
| embedding (Embedding)         | (None, 128, 300)          |       5,611,800 | input_layer[0][0]          |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv1d (Conv1D)               | (None, 128, 128)          |          38,528 | embedding[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv1d_1 (Conv1D)             | (None, 127, 128)          |          76,928 | embedding[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv1d_2 (Conv1D)             | (None, 126, 128)          |         115,328 | embedding[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv1d_3 (Conv1D)             | (None, 124, 128)          |         192,128 | embedding[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| conv1d_4 (Conv1D)             | (None, 123, 128)          |         230,528 | embedding[0][0]            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| dropout (Dropout)             | (None, 128, 128)          |               0 | conv1d[0][0]               |
|-------------------------------+---------------------------+-----------------+----------------------------|
| dropout_2 (Dropout)           | (None, 127, 128)          |               0 | conv1d_1[0][0]             |
|-------------------------------+---------------------------+-----------------+----------------------------|
| dropout_4 (Dropout)           | (None, 126, 128)          |               0 | conv1d_2[0][0]             |
|-------------------------------+---------------------------+-----------------+----------------------------|
| dropout_6 (Dropout)           | (None, 124, 128)          |               0 | conv1d_3[0][0]             |
|-------------------------------+---------------------------+-----------------+----------------------------|
| dropout_8 (Dropout)           | (None, 123, 128)          |               0 | conv1d_4[0][0]             |
|-------------------------------+---------------------------+-----------------+----------------------------|
| max_pooling1d (MaxPooling1D)  | (None, 32, 128)           |               0 | dropout[0][0]              |
|-------------------------------+---------------------------+-----------------+----------------------------|
| max_pooling1d_1               | (None, 31, 128)           |               0 | dropout_2[0][0]            |
| (MaxPooling1D)                |                           |                 |                            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| max_pooling1d_2               | (None, 31, 128)           |               0 | dropout_4[0][0]            |
| (MaxPooling1D)                |                           |                 |                            |
|-------------------------------+---------------------------+-----------------+----------------------------|
| max_pooling1d_3               | (None, 31, 128)           |               0 | dropout_6[0][0]            |
| (MaxPooling1D)                |                           |               

 Total params: 12,257,241 (46.76 MB)

 Trainable params: 6,641,441 (25.34 MB)

...

Fold: 01 out of 05 completed.
Local System Time: 12:20 PM
2025-05-23 12:20:14
Pretrained Embeddings: 14132 hits (4675 misses)
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 135ms/step


Epoch: 001 --LR: 1e-03 --MaxValF1: 0.8531547 --CurValF1: 0.8531547 --Patience: 01 --F1 improved: 0.8531547
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step
Epoch: 002 --LR: 1e-03 --MaxValF1: 0.8531547 --CurValF1: 0.8519934 --Patience: 01
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step
Epoch: 003 --LR: 3e-04 --MaxValF1: 0.8531547 --CurValF1: 0.8529251 --Patience: 02
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step
Epoch: 004 --LR: 9e-05 --MaxValF1: 0.8531547 --CurValF1: 0.8529412 --Patience: 03
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step


Epoch: 005 --LR: 3e-05 --MaxValF1: 0.8535179 --CurValF1: 0.8535179 --Patience: 04 --F1 improved: 0.8535179
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step
Epoch: 006 --LR: 3e-05 --MaxValF1: 0.8535179 --CurValF1: 0.8529251 --Patience: 01
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step


Epoch: 007 --LR: 8e-06 --MaxValF1: 0.8538813 --CurValF1: 0.8538813 --Patience: 02 --F1 improved: 0.8538813


KeyboardInterrupt: 

In [ ]:
# hyper parameters for this model
MAX_LEN = 100
EMBED_SIZE = 300
EMBED_TRAINABLE = False
EMB_WEIGHTS_INIT = 'he_normal'
SPDRPT = 0.3
DRPT = 0.1
CONV_WEIGHTS_INIT = 'he_uniform'
CONV_ACT = 'elu'
FC_WEIGHTS_INIT = 'he_uniform'
FC_ACT = 'elu'
LR_RATE = 1e-3
OPTIMIZER = 'adam'
BATCH = 32
NEPOCHS = 30
PATIENCE = 5
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'CNN_RUT-ft'
modelpath = './Saved Models/{}/'.format( modelname )
modelresults = './Model Results'
modelsummaries = './Model - Summaries-Figures'
DIRECTORIES_TO_BE_CREATED = [ modelpath, modelresults, modelsummaries ]
for directory in DIRECTORIES_TO_BE_CREATED:
    if not os.path.exists( directory ):os.makedirs( directory )

if PRE_TRAINED_FLAG == True:
    EMBEDDINGS_INDEX = dict( get_coefs( *o.strip().split() ) for o in open( EMBEDDING_FILE, encoding='utf-8' )  if len(o)>19 )
    print( 'Total Words in Pretrained Embeddings: ', len( EMBEDDINGS_INDEX.values() ) )
    all_embs = np.stack( list(EMBEDDINGS_INDEX.values()) )
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    print( 'Pretrained Embeddings mean:', emb_mean, ' std: ', emb_std )

def CNN_RUT(
    max_len=100, embed_inp=10000, embed_size=300, embedding_matrix=[], embed_trainable=False,
    spdrpt=.1, drpt=.1, emb_weights_init='', conv_weights_init='', conv_act='', fc_weights_init='',
    fc_act='', optimizer='' ):

    filter_sizes = [ 1,2,3,4,5 ]
    num_filters = 32

    inp = Input( shape=( max_len, ) )
    if embedding_matrix is None:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      embeddings_initializer=emb_weights_init, trainable=embed_trainable )( inp )
    else:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      weights=[ embedding_matrix ], trainable=embed_trainable )( inp )

    x = SpatialDropout1D( spdrpt )( x )
    x = Reshape( ( max_len, embed_size, 1 ) )( x )
    pooled = [  ]
    for j,i in enumerate( filter_sizes ):
        conv = Conv2D( num_filters, kernel_size=( i, embed_size ), activation=conv_act )( x )
        globalmax = GlobalMaxPooling2D(  )( conv )
        pooled.append( globalmax )
    z = Concatenate( axis=1 )( pooled )
    z = Dropout( drpt )( z )
    fc1 = Dense( 100, activation=fc_act, kernel_initializer=fc_weights_init )( z )
    fc2 = Dense( 50, activation=fc_act, kernel_initializer=fc_weights_init )( fc1 )
    outp = Dense( 1, activation='sigmoid' )( fc2 )

    model = Model( inputs=inp, outputs=outp )

    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model

In [ ]:
warnings.filterwarnings( 'ignore' )
start_time = time.time()
print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
com_text, com_label, com_predicted, com_prob = [], [], [], []
reports = []
hist = {}
skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
print(skf)
fold = 1
for train_index, test_index in skf.split( df[ xcolumn ], df[ ycolumn ] ):
    # seprating train and test sets
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    xtrain = df.loc[ train_index ][ xcolumn ].values
    xtest = df.loc[ test_index ][ xcolumn ].values

    ytrain = df.loc[ train_index ][ ycolumn ].values
    ytest = df.loc[ test_index ][ ycolumn ].values

    # splitting train and validation sets
    xtrain, xval, ytrain, yval = train_test_split( xtrain, ytrain, test_size=0.15, random_state=0 )

    # tokenization with keras tokenizer
    tokenizer = text.Tokenizer(  )
    tokenizer.fit_on_texts( np.append( xtrain, xval ) )

    xtrain = tokenizer.texts_to_sequences( xtrain )
    xval = tokenizer.texts_to_sequences( xval )
    xtest = tokenizer.texts_to_sequences( xtest )

    # pad the tokenized sequences
    xtrain = sequence.pad_sequences( xtrain, maxlen=MAX_LEN )
    xval = sequence.pad_sequences( xval, maxlen=MAX_LEN )
    xtest = sequence.pad_sequences( xtest, maxlen=MAX_LEN )

    # check if pre-trained word embeddings flag is true
    if PRE_TRAINED_FLAG == True:
        EMBEDDING_MATRIX = get_vectors(tokenizer=tokenizer, emb_mean=emb_mean, emb_std=emb_std, embed_size=EMBED_SIZE)

    # define a model
    EMBED_INP_SIZE = len( tokenizer.word_index ) + 1
    model = CNN_RUT(max_len=MAX_LEN, embed_inp=EMBED_INP_SIZE, embed_size=EMBED_SIZE,
        embedding_matrix=EMBEDDING_MATRIX, embed_trainable=EMBED_TRAINABLE, spdrpt=SPDRPT,
        drpt=DRPT, emb_weights_init=EMB_WEIGHTS_INIT, conv_weights_init=CONV_WEIGHTS_INIT,
        conv_act=CONV_ACT, fc_weights_init=FC_WEIGHTS_INIT, fc_act=FC_ACT, optimizer=OPTIMIZER     )

    # save model summaries and model architecture diagrams in the first fold only
    if fold == 1:
        #plot_model( model=model, to_file='{}/{}.png'.format( modelsummaries, modelname ), show_shapes=False )

        with open( '{}/{}.txt'.format( modelsummaries, modelname ), 'w' ) as s:
            with redirect_stdout( s ):model.summary()

    if isinstance(model.optimizer.learning_rate, tf.Variable):
      K.set_value(model.optimizer.learning_rate, LR_RATE)
    else:
      model.optimizer.learning_rate = LR_RATE
    modelpath = modelpath + modelname + str( fold ) + ".keras"

    # train the model with callbacks for early stopping
    f1callback = F1_score_callback(
        val_data=( xval, yval ), filepath=modelpath,
        patience=PATIENCE, decay=DECAY, decay_rate=DECAY_RATE, decay_after=DECAY_AFTER
    )
    histmodel = model.fit(
        xtrain, ytrain, batch_size=BATCH, epochs=NEPOCHS, verbose=0, callbacks=[ f1callback ]
    )

    # save history of all folds
    hist[ 'fold' + str( fold ) ] = histmodel.history.copy()

    # delete trained model object
    del model, histmodel, f1callback
    EMBEDDING_MATRIX = []
    K.clear_session()
    gc.collect()
    # delete pre trained embeddings from the memory
    if ( PRE_TRAINED_FLAG == True ) and ( fold == skf.n_splits ):
        del EMBEDDINGS_INDEX
        gc.collect()

    # load saved model
    loaded_model = load_model( modelpath )

    # get predictions (probabilities) for validation and test sets respectively
    valpredictions = loaded_model.predict( xval, verbose=0, batch_size=BATCH )
    testpredictions = loaded_model.predict( xtest, verbose=0, batch_size=BATCH )

    # delete loaded model
    del loaded_model
    K.clear_session()
    gc.collect()

    # optimizer threshold on validation set
    threshold = optimize_threshold( yval, valpredictions )

    # save accuracy, precision, recall, f1 and confusion matrices
    vallabels = ( valpredictions >= threshold ).astype( 'int32' )
    testlabels = ( testpredictions >= threshold ).astype( 'int32' )

    valaccuracy.append( accuracy_score( yval, vallabels ) )
    valprecision.append( precision_score( yval, vallabels ) )
    valrecall.append( recall_score( yval, vallabels ) )
    valf1.append( f1_score( yval, vallabels ) )
    valcm.append( confusion_matrix( yval, vallabels ) )

    testaccuracy.append( accuracy_score( ytest, testlabels ) )
    testprecision.append( precision_score( ytest, testlabels ) )
    testrecall.append( recall_score( ytest, testlabels ) )
    testf1.append( f1_score( ytest, testlabels ) )
    testcm.append( confusion_matrix( ytest, testlabels ) )
    reports.append(classification_report( ytest, testlabels, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))

    print( 'Fold: {:02d} out of {:02d} completed.'.format( fold, skf.get_n_splits() ) )
    print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

    fold = fold + 1
time_took = time.time() - start_time
print( f"Total runtime: { hms_string( time_took ) }" )
WriteResutls(reports)

In [ ]:
# hyper parameters for this model

MAX_LEN = 128
EMBED_SIZE = 300
EMBED_TRAINABLE = False
EMB_WEIGHTS_INIT = 'he_normal'
SPDRPT = 0.4
DRPT = 0.2
FC_WEIGHTS_INIT = 'he_uniform'
FC_ACT = 'elu'
LR_RATE = 1e-3
OPTIMIZER = 'adam'
GRU_UNITS = 100
BATCH = 32
NEPOCHS = 30
PATIENCE = 5
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'BGRU-ft'
modelpath = './Saved Models/' + modelname + '/'
modelresults = './Model Results'
modelsummaries = './Model - Summaries-Figures'
DIRECTORIES_TO_BE_CREATED = [ modelpath, modelresults, modelsummaries ]
for directory in DIRECTORIES_TO_BE_CREATED:
    if not os.path.exists( directory ):os.makedirs( directory )

if PRE_TRAINED_FLAG == True:
    EMBEDDINGS_INDEX = dict( get_coefs( *o.strip().split() ) for o in open( EMBEDDING_FILE, encoding='utf-8' )  if len(o)>19 )
    print( 'Total Words in Pretrained Embeddings: ', len( EMBEDDINGS_INDEX.values() ) )
    all_embs = np.stack( list(EMBEDDINGS_INDEX.values()) )
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    print( 'Pretrained Embeddings mean:', emb_mean, ' std: ', emb_std )

def BGRU(
    max_len=100, embed_inp=10000, embed_size=300, embedding_matrix=[], embed_trainable=False,
    spdrpt=.1, drpt=.1, emb_weights_init='', fc_weights_init='', fc_act='', optimizer='', gru_units=32 ):

    inp = Input( shape=( max_len, ) )
    if embedding_matrix is None:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      embeddings_initializer=emb_weights_init, trainable=embed_trainable )( inp )
    else:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      weights=[ embedding_matrix ], trainable=embed_trainable )( inp )

    x = SpatialDropout1D( spdrpt )( x )
    z = Bidirectional( GRU( gru_units ) )( x )
    z = Dropout( drpt )( z )
    fc1 = Dense( 100, activation=fc_act, kernel_initializer=fc_weights_init )( z )
    fc2 = Dense( 50, activation=fc_act, kernel_initializer=fc_weights_init )( fc1 )
    output = Dense( 1, activation='sigmoid' )( fc2 )
    model = Model( inputs=inp, outputs=output )

    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model


In [ ]:
warnings.filterwarnings( 'ignore' )
start_time = time.time()
print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
com_text, com_label, com_predicted, com_prob = [], [], [], []
com_indices = []
hist = {}
reports = []
skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
print(skf)
fold = 1
for train_index, test_index in skf.split( df[ xcolumn ], df[ ycolumn ] ):
    # seprating train and test sets
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    xtrain = df.loc[ train_index ][ xcolumn ].values
    xtest = df.loc[ test_index ][ xcolumn ].values

    ytrain = df.loc[ train_index ][ ycolumn ].values
    ytest = df.loc[ test_index ][ ycolumn ].values

    # splitting train and validation sets
    xtrain, xval, ytrain, yval = train_test_split( xtrain, ytrain, test_size=0.15, random_state=0 )

    # tokenization with keras tokenizer
    tokenizer = text.Tokenizer(  )
    tokenizer.fit_on_texts( np.append( xtrain, xval ) )

    xtrain = tokenizer.texts_to_sequences( xtrain )
    xval = tokenizer.texts_to_sequences( xval )
    xtest = tokenizer.texts_to_sequences( xtest )

    # pad the tokenized sequences
    xtrain = sequence.pad_sequences( xtrain, maxlen=MAX_LEN )
    xval = sequence.pad_sequences( xval, maxlen=MAX_LEN )
    xtest = sequence.pad_sequences( xtest, maxlen=MAX_LEN )

    # check if pre-trained word embeddings flag is true
    if PRE_TRAINED_FLAG == True:
        EMBEDDING_MATRIX = get_vectors(tokenizer=tokenizer, emb_mean=emb_mean, emb_std=emb_std, embed_size=EMBED_SIZE)

    # define a model
    EMBED_INP_SIZE = len( tokenizer.word_index ) + 1
    model = BGRU( max_len=MAX_LEN, embed_inp=EMBED_INP_SIZE, embed_size=EMBED_SIZE,
        embedding_matrix=EMBEDDING_MATRIX, embed_trainable=EMBED_TRAINABLE, spdrpt=SPDRPT, drpt=DRPT, emb_weights_init=EMB_WEIGHTS_INIT,
        fc_weights_init=FC_WEIGHTS_INIT, fc_act=FC_ACT, optimizer=OPTIMIZER, gru_units=GRU_UNITS )

    # save model summaries and model architecture diagrams in the first fold only
    if fold == 1:
        plot_model( model=model, to_file='{}/{}.png'.format( modelsummaries, modelname ), show_shapes=False )

        with open( '{}/{}.txt'.format( modelsummaries, modelname ), 'w' ) as s:
            with redirect_stdout( s ): model.summary()

    if isinstance(model.optimizer.learning_rate, tf.Variable):
      K.set_value(model.optimizer.learning_rate, LR_RATE)
    else:
      model.optimizer.learning_rate = LR_RATE
    modelpath = modelpath + modelname + str( fold ) + ".keras"
    # train the model with callbacks for early stopping
    f1callback = F1_score_callback( val_data=( xval, yval ), filepath=modelpath,
        patience=PATIENCE, decay=DECAY, decay_rate=DECAY_RATE, decay_after=DECAY_AFTER  )
    histmodel = model.fit( xtrain, ytrain, batch_size=BATCH, epochs=NEPOCHS, verbose=0, callbacks=[ f1callback ] )

    # save history of all folds
    hist[ 'fold' + str( fold ) ] = histmodel.history.copy()

    # delete trained model object
    del model, histmodel, f1callback
    EMBEDDING_MATRIX = []
    K.clear_session()
    gc.collect()
    # delete pre trained embeddings from the memory
    if ( PRE_TRAINED_FLAG == True ) and ( fold == skf.n_splits ):
        del EMBEDDINGS_INDEX
        gc.collect()

    # load saved model
    loaded_model = load_model( modelpath )

    # get predictions (probabilities) for validation and test sets respectively
    valpredictions = loaded_model.predict( xval, verbose=0, batch_size=BATCH )
    testpredictions = loaded_model.predict( xtest, verbose=0, batch_size=BATCH )

    # delete loaded model
    del loaded_model
    K.clear_session()
    gc.collect()

    # optimizer threshold on validation set
    threshold = optimize_threshold( yval, valpredictions )

    # save accuracy, precision, recall, f1 and confusion matrices
    vallabels = ( valpredictions >= threshold ).astype( 'int32' )
    testlabels = ( testpredictions >= threshold ).astype( 'int32' )

    valaccuracy.append( accuracy_score( yval, vallabels ) )
    valprecision.append( precision_score( yval, vallabels ) )
    valrecall.append( recall_score( yval, vallabels ) )
    valf1.append( f1_score( yval, vallabels ) )
    valcm.append( confusion_matrix( yval, vallabels ) )

    testaccuracy.append( accuracy_score( ytest, testlabels ) )
    testprecision.append( precision_score( ytest, testlabels ) )
    testrecall.append( recall_score( ytest, testlabels ) )
    testf1.append( f1_score( ytest, testlabels ) )
    testcm.append( confusion_matrix( ytest, testlabels ) )
    reports.append(classification_report( ytest, testlabels, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))
    print( 'Fold: {:02d} out of {:02d} completed.'.format( fold, skf.get_n_splits() ) )
    print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

    fold = fold + 1
time_took = time.time() - start_time
print( f"Total runtime: { hms_string( time_took ) }" )
WriteResutls(reports)

In [ ]:
# hyper parameters for this model

MAX_LEN = 128
EMBED_SIZE = 300
EMBED_TRAINABLE = False
EMB_WEIGHTS_INIT = 'he_normal'
SPDRPT = 0.4
DRPT = 0.2
FC_WEIGHTS_INIT = 'he_uniform'
FC_ACT = 'elu'
LR_RATE = 1e-3
OPTIMIZER = 'adam'
LSTM_UNITS = 100
BATCH = 32
NEPOCHS = 30
PATIENCE = 5
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'BLSTM-ft'
modelpath = './Saved Models/' + modelname + '/'
modelresults = './Model Results'
modelsummaries = './Model - Summaries-Figures'
DIRECTORIES_TO_BE_CREATED = [ modelpath, modelresults, modelsummaries ]
for directory in DIRECTORIES_TO_BE_CREATED:
    if not os.path.exists( directory ): os.makedirs( directory )

if PRE_TRAINED_FLAG == True:
    EMBEDDINGS_INDEX = dict( get_coefs( *o.strip().split() ) for o in open( EMBEDDING_FILE, encoding='utf-8' )  if len(o)>19 )
    print( 'Total Words in Pretrained Embeddings: ', len( EMBEDDINGS_INDEX.values() ) )
    all_embs = np.stack( list(EMBEDDINGS_INDEX.values()) )
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    print( 'Pretrained Embeddings mean:', emb_mean, ' std: ', emb_std )

def BLSTM(
    max_len=100, embed_inp=10000, embed_size=300, embedding_matrix=[], embed_trainable=False,
    spdrpt=.1, drpt=.1, emb_weights_init='', fc_weights_init='', fc_act='', optimizer='', lstm_units=32 ):

    inp = Input( shape=( max_len, ) )
    if embedding_matrix is None:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      embeddings_initializer=emb_weights_init, trainable=embed_trainable )( inp )
    else:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      weights=[ embedding_matrix ], trainable=embed_trainable )( inp )

    x = SpatialDropout1D( spdrpt )( x )
    z = Bidirectional( LSTM( lstm_units ) )( x )
    z = Dropout( drpt )( z )
    fc1 = Dense( 100, activation=fc_act, kernel_initializer=fc_weights_init )( z )
    fc2 = Dense( 50, activation=fc_act, kernel_initializer=fc_weights_init )( fc1 )
    output = Dense( 1, activation='sigmoid' )( fc2 )

    model = Model( inputs=inp, outputs=output )

    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model

In [ ]:
warnings.filterwarnings( 'ignore' )
start_time = time.time()

valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
com_text, com_label, com_predicted, com_prob = [], [], [], []

skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
print(skf)
reports = []
fold = 1
for train_index, test_index in skf.split( df[ xcolumn ], df[ ycolumn ] ):
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    # seprating train and test sets
    xtrain = df.loc[ train_index ][ xcolumn ].values
    xtest = df.loc[ test_index ][ xcolumn ].values

    ytrain = df.loc[ train_index ][ ycolumn ].values
    ytest = df.loc[ test_index ][ ycolumn ].values

    # splitting train and validation sets
    xtrain, xval, ytrain, yval = train_test_split( xtrain, ytrain, test_size=0.15, random_state=0 )

    # tokenization with keras tokenizer
    tokenizer = text.Tokenizer(  )
    tokenizer.fit_on_texts( np.append( xtrain, xval ) )

    xtrain = tokenizer.texts_to_sequences( xtrain )
    xval = tokenizer.texts_to_sequences( xval )
    xtest = tokenizer.texts_to_sequences( xtest )

    # pad the tokenized sequences
    xtrain = sequence.pad_sequences( xtrain, maxlen=MAX_LEN )
    xval = sequence.pad_sequences( xval, maxlen=MAX_LEN )
    xtest = sequence.pad_sequences( xtest, maxlen=MAX_LEN )

    # check if pre-trained word embeddings flag is true
    if PRE_TRAINED_FLAG == True:
        EMBEDDING_MATRIX = get_vectors(
            tokenizer=tokenizer, emb_mean=emb_mean, emb_std=emb_std, embed_size=EMBED_SIZE
        )

    # define a model
    EMBED_INP_SIZE = len( tokenizer.word_index ) + 1
    model = BLSTM(
        max_len=MAX_LEN, embed_inp=EMBED_INP_SIZE, embed_size=EMBED_SIZE,
        embedding_matrix=EMBEDDING_MATRIX, embed_trainable=EMBED_TRAINABLE,
        spdrpt=SPDRPT, drpt=DRPT, emb_weights_init=EMB_WEIGHTS_INIT,
        fc_weights_init=FC_WEIGHTS_INIT, fc_act=FC_ACT, optimizer=OPTIMIZER, lstm_units=LSTM_UNITS
    )

    # save model summaries and model architecture diagrams in the first fold only
    if fold == 1:
        plot_model( model=model, to_file='{}/{}.png'.format( modelsummaries, modelname ), show_shapes=False )

        with open( '{}/{}.txt'.format( modelsummaries, modelname ), 'w' ) as s:
            with redirect_stdout( s ):
                model.summary()

    if isinstance(model.optimizer.learning_rate, tf.Variable):
      K.set_value(model.optimizer.learning_rate, LR_RATE)
    else:
      model.optimizer.learning_rate = LR_RATE

    # train the model with callbacks for early stopping
    f1callback = F1_score_callback(
        val_data=( xval, yval ), filepath=modelpath + modelname + str( fold )+".keras",
        patience=PATIENCE, decay=DECAY, decay_rate=DECAY_RATE, decay_after=DECAY_AFTER
    )
    histmodel = model.fit(
        xtrain, ytrain, batch_size=BATCH, epochs=NEPOCHS, verbose=0, callbacks=[ f1callback ]
    )

    # save history of all folds
    hist[ 'fold' + str( fold ) ] = histmodel.history.copy()

    # delete trained model object
    del model, histmodel, f1callback
    EMBEDDING_MATRIX = []
    K.clear_session()
    gc.collect()
    # delete pre trained embeddings from the memory
    if ( PRE_TRAINED_FLAG == True ) and ( fold == skf.n_splits ):
        del EMBEDDINGS_INDEX
        gc.collect()

    # load saved model
    loaded_model = load_model( modelpath + modelname + str( fold ) + ".keras" )

    # get predictions (probabilities) for validation and test sets respectively
    valpredictions = loaded_model.predict( xval, verbose=0, batch_size=BATCH )
    testpredictions = loaded_model.predict( xtest, verbose=0, batch_size=BATCH )

    # delete loaded model
    del loaded_model
    K.clear_session()
    gc.collect()

    # optimizer threshold on validation set
    threshold = optimize_threshold( yval, valpredictions )

    # save accuracy, precision, recall, f1 and confusion matrices
    vallabels = ( valpredictions >= threshold ).astype( 'int32' )
    testlabels = ( testpredictions >= threshold ).astype( 'int32' )

    valaccuracy.append( accuracy_score( yval, vallabels ) )
    valprecision.append( precision_score( yval, vallabels ) )
    valrecall.append( recall_score( yval, vallabels ) )
    valf1.append( f1_score( yval, vallabels ) )
    valcm.append( confusion_matrix( yval, vallabels ) )

    testaccuracy.append( accuracy_score( ytest, testlabels ) )
    testprecision.append( precision_score( ytest, testlabels ) )
    testrecall.append( recall_score( ytest, testlabels ) )
    testf1.append( f1_score( ytest, testlabels ) )
    testcm.append( confusion_matrix( ytest, testlabels ) )
    reports.append(classification_report( ytest, testlabels, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))
    print( 'Fold: {:02d} out of {:02d} completed.'.format( fold, skf.get_n_splits() ) )
    print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

    fold = fold + 1
time_took = time.time() - start_time
print( f"Total runtime: { hms_string( time_took ) }" )
WriteResutls(reports)

In [ ]:
# hyper parameters for this model

MAX_LEN = 128
EMBED_SIZE = 300
EMBED_TRAINABLE = False
EMB_WEIGHTS_INIT = 'he_normal'
SPDRPT = 0.35
DRPT = 0.2
FC_WEIGHTS_INIT = 'he_uniform'
FC_ACT = 'elu'
LR_RATE = 1e-3
OPTIMIZER = 'adam'
FCL_LOSS_ALP = 0.25
FCL_LOSS_GAM = 5
BATCH = 32
NEPOCHS = 30
PATIENCE = 5
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'BGRU_P-ft'
modelpath = './Saved Models/' + modelname + '/'
modelresults = './Model Results'
modelsummaries = './Model - Summaries-Figures'
DIRECTORIES_TO_BE_CREATED = [ modelpath, modelresults, modelsummaries ]
for directory in DIRECTORIES_TO_BE_CREATED:
    if not os.path.exists( directory ):
        os.makedirs( directory )

if PRE_TRAINED_FLAG == True:
    EMBEDDINGS_INDEX = dict( get_coefs( *o.strip().split() ) for o in open( EMBEDDING_FILE, encoding='utf-8' )  if len(o)>19 )
    print( 'Total Words in Pretrained Embeddings: ', len( EMBEDDINGS_INDEX.values() ) )
    all_embs = np.stack( list(EMBEDDINGS_INDEX.values()) )
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    print( 'Pretrained Embeddings mean:', emb_mean, ' std: ', emb_std )

def BGRU_P(
    max_len=100, embed_inp=10000, embed_size=300, embedding_matrix=[], embed_trainable=False,
    spdrpt=.1, drpt=.1, emb_weights_init='', fc_weights_init='', fc_act='', optimizer='',
    fcl_loss_alp=.1, fcl_loss_gam=1 ):

    inp = Input( shape=( max_len, ) )
    if embedding_matrix is None:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      embeddings_initializer=emb_weights_init, trainable=embed_trainable )( inp )
    else:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      weights=[ embedding_matrix ], trainable=embed_trainable )( inp )

    x = SpatialDropout1D( spdrpt )( x )
    x1 = Bidirectional( GRU( 128, return_sequences=True ) )(x)
    x2 = Bidirectional( GRU( 64, return_sequences=True ) )(x)

    conc = concatenate( [ x1, x2 ] )

    pooled = [  ]

    avg_pool = GlobalAveragePooling1D(  )(conc)
    max_pool = GlobalMaxPooling1D(  )(conc)

    pooled.append( avg_pool )
    pooled.append( max_pool )

    x = Concatenate( axis=1 )( pooled )
    x = Dropout( drpt ) ( x )

    fc1 = Dense( 100, activation=fc_act, kernel_initializer=fc_weights_init )( x )
    fc2 = Dense( 50, activation=fc_act, kernel_initializer=fc_weights_init )( fc1 )
    output = Dense( 1, activation='sigmoid' )( fc2 )

    model = Model( inputs=inp, outputs=output )

    model.compile(loss=[ focal_loss( alpha=fcl_loss_alp, gamma=fcl_loss_gam ) ],
        optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
warnings.filterwarnings( 'ignore' )
start_time = time.time()
print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
com_text, com_label, com_predicted, com_prob = [], [], [], []
com_indices = []
hist = {}
reports = []
skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
print(skf)

fold = 1
for train_index, test_index in skf.split( df[ xcolumn ], df[ ycolumn ] ):
    # seprating train and test sets
    xtrain = df.loc[ train_index ][ xcolumn ].values
    xtest = df.loc[ test_index ][ xcolumn ].values

    ytrain = df.loc[ train_index ][ ycolumn ].values
    ytest = df.loc[ test_index ][ ycolumn ].values

    # splitting train and validation sets
    xtrain, xval, ytrain, yval = train_test_split( xtrain, ytrain, test_size=0.15, random_state=0 )

    # tokenization with keras tokenizer
    tokenizer = text.Tokenizer(  )
    tokenizer.fit_on_texts( np.append( xtrain, xval ) )

    xtrain = tokenizer.texts_to_sequences( xtrain )
    xval = tokenizer.texts_to_sequences( xval )
    xtest = tokenizer.texts_to_sequences( xtest )

    # pad the tokenized sequences
    xtrain = sequence.pad_sequences( xtrain, maxlen=MAX_LEN )
    xval = sequence.pad_sequences( xval, maxlen=MAX_LEN )
    xtest = sequence.pad_sequences( xtest, maxlen=MAX_LEN )

    # check if pre-trained word embeddings flag is true
    if PRE_TRAINED_FLAG == True:
        EMBEDDING_MATRIX = get_vectors(
            tokenizer=tokenizer, emb_mean=emb_mean, emb_std=emb_std, embed_size=EMBED_SIZE
        )

    # define a model
    EMBED_INP_SIZE = len( tokenizer.word_index ) + 1
    model = BGRU_P(
        max_len=MAX_LEN, embed_inp=EMBED_INP_SIZE, embed_size=EMBED_SIZE,
        embedding_matrix=EMBEDDING_MATRIX, embed_trainable=EMBED_TRAINABLE,
        spdrpt=SPDRPT, drpt=DRPT, emb_weights_init=EMB_WEIGHTS_INIT, fc_weights_init=FC_WEIGHTS_INIT,
        fc_act=FC_ACT, optimizer=OPTIMIZER, fcl_loss_alp=FCL_LOSS_ALP, fcl_loss_gam=FCL_LOSS_GAM
    )

    # save model summaries and model architecture diagrams in the first fold only
    if fold == 1:
        plot_model( model=model, to_file='{}/{}.png'.format( modelsummaries, modelname ), show_shapes=False )

        with open( '{}/{}.txt'.format( modelsummaries, modelname ), 'w' ) as s:
            with redirect_stdout( s ):
                model.summary()

    if isinstance(model.optimizer.learning_rate, tf.Variable):
      K.set_value(model.optimizer.learning_rate, LR_RATE)
    else:
      model.optimizer.learning_rate = LR_RATE

    # train the model with callbacks for early stopping
    f1callback = F1_score_callback(
        val_data=( xval, yval ), filepath=modelpath + modelname + str( fold ) + ".keras",
        patience=PATIENCE, decay=DECAY, decay_rate=DECAY_RATE, decay_after=DECAY_AFTER
    )
    histmodel = model.fit(
        xtrain, ytrain, batch_size=BATCH, epochs=NEPOCHS, verbose=0, callbacks=[ f1callback ]
    )

    # save history of all folds
    hist[ 'fold' + str( fold ) ] = histmodel.history.copy()

    # delete trained model object
    del model, histmodel, f1callback
    EMBEDDING_MATRIX = []
    K.clear_session()
    gc.collect()
    # delete pre trained embeddings from the memory
    if ( PRE_TRAINED_FLAG == True ) and ( fold == skf.n_splits ):
        del EMBEDDINGS_INDEX
        gc.collect()

    # load saved model
    custt_obss = { 'focal_loss_fixed' : focal_loss( alpha=FCL_LOSS_ALP, gamma=FCL_LOSS_GAM ) }
    loaded_model = load_model( modelpath + modelname + str( fold )+".keras", custom_objects=custt_obss )

    # get predictions (probabilities) for validation and test sets respectively
    valpredictions = loaded_model.predict( xval, verbose=0, batch_size=BATCH )
    testpredictions = loaded_model.predict( xtest, verbose=0, batch_size=BATCH )

    # delete loaded model
    del loaded_model, custt_obss
    K.clear_session()
    gc.collect()

    # optimizer threshold on validation set
    threshold = optimize_threshold( yval, valpredictions )

    # save accuracy, precision, recall, f1 and confusion matrices
    vallabels = ( valpredictions >= threshold ).astype( 'int32' )
    testlabels = ( testpredictions >= threshold ).astype( 'int32' )

    valaccuracy.append( accuracy_score( yval, vallabels ) )
    valprecision.append( precision_score( yval, vallabels ) )
    valrecall.append( recall_score( yval, vallabels ) )
    valf1.append( f1_score( yval, vallabels ) )
    valcm.append( confusion_matrix( yval, vallabels ) )

    testaccuracy.append( accuracy_score( ytest, testlabels ) )
    testprecision.append( precision_score( ytest, testlabels ) )
    testrecall.append( recall_score( ytest, testlabels ) )
    testf1.append( f1_score( ytest, testlabels ) )
    testcm.append( confusion_matrix( ytest, testlabels ) )
    reports.append(classification_report( ytest, testlabels, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))

    print( 'Fold: {:02d} out of {:02d} completed.'.format( fold, skf.get_n_splits() ) )
    print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

    fold = fold + 1
time_took = time.time() - start_time
print( f"Total runtime: { hms_string( time_took ) }" )
WriteResutls(reports)

In [ ]:
# hyper parameters for this model

MAX_LEN = 128
EMBED_SIZE = 300
EMBED_TRAINABLE = False
EMB_WEIGHTS_INIT = 'he_normal'
LR_RATE = 1e-3
OPTIMIZER = 'adam'
KER_REGULARIZER = 'L1L2'
DRPT = 0.3

BATCH = 32
NEPOCHS = 30
PATIENCE = 5
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'CNN_GRU-ft'
modelpath = './Saved Models/' + modelname + '/'
modelresults = './Model Results'
modelsummaries = './Model - Summaries-Figures'

DIRECTORIES_TO_BE_CREATED = [ modelpath, modelresults, modelsummaries ]
for directory in DIRECTORIES_TO_BE_CREATED:
    if not os.path.exists( directory ):
        os.makedirs( directory )

if PRE_TRAINED_FLAG == True:
    EMBEDDINGS_INDEX = dict( get_coefs( *o.strip().split() ) for o in open( EMBEDDING_FILE, encoding='utf-8' )  if len(o)>19 )
    print( 'Total Words in Pretrained Embeddings: ', len( EMBEDDINGS_INDEX.values() ) )
    all_embs = np.stack( list(EMBEDDINGS_INDEX.values()) )
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    print( 'Pretrained Embeddings mean:', emb_mean, ' std: ', emb_std )

def CNN_GRU(
    max_len=100, embed_inp=10000, embed_size=300, embedding_matrix=[], embed_trainable=False,
    drpt=.1, emb_weights_init='', optimizer='', ker_regularizer='' ):

    inp = Input( shape=( max_len, ) )
    if embedding_matrix is None:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      embeddings_initializer=emb_weights_init, trainable=embed_trainable )( inp )
    else:
        x = Embedding( input_dim=embed_inp, output_dim=embed_size,
                      weights=[ embedding_matrix ], trainable=embed_trainable )( inp )

    x = Dropout( drpt )( x )
    conv = Conv1D( filters=100, kernel_size=4, padding='same', activation='relu',
                  kernel_regularizer=ker_regularizer )( x )
    pool = MaxPooling1D( pool_size=4 )( conv )
    gru = GRU( 100, return_sequences=True )( pool )
    z = GlobalMaxPooling1D(  )( gru )
    output = Dense( 1, activation='sigmoid', kernel_regularizer=ker_regularizer )( z )

    model = Model( inputs=inp, outputs=output )

    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model

In [ ]:
warnings.filterwarnings( 'ignore' )
start_time = time.time()
print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
com_text, com_label, com_predicted, com_prob = [], [], [], []
com_indices = []
hist = {}
reports = []
skf = StratifiedKFold( n_splits=5, random_state=0, shuffle=True )
print(skf)

fold = 1
for train_index, test_index in skf.split( df[ xcolumn ], df[ ycolumn ] ):
    # seprating train and test sets
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    xtrain = df.loc[ train_index ][ xcolumn ].values
    xtest = df.loc[ test_index ][ xcolumn ].values

    ytrain = df.loc[ train_index ][ ycolumn ].values
    ytest = df.loc[ test_index ][ ycolumn ].values

    # splitting train and validation sets
    xtrain, xval, ytrain, yval = train_test_split( xtrain, ytrain, test_size=0.15, random_state=0 )

    # tokenization with keras tokenizer
    tokenizer = text.Tokenizer(  )
    tokenizer.fit_on_texts( np.append( xtrain, xval ) )

    xtrain = tokenizer.texts_to_sequences( xtrain )
    xval = tokenizer.texts_to_sequences( xval )
    xtest = tokenizer.texts_to_sequences( xtest )

    # pad the tokenized sequences
    xtrain = sequence.pad_sequences( xtrain, maxlen=MAX_LEN )
    xval = sequence.pad_sequences( xval, maxlen=MAX_LEN )
    xtest = sequence.pad_sequences( xtest, maxlen=MAX_LEN )

    # check if pre-trained word embeddings flag is true
    if PRE_TRAINED_FLAG == True:
        EMBEDDING_MATRIX = get_vectors(
            tokenizer=tokenizer, emb_mean=emb_mean, emb_std=emb_std, embed_size=EMBED_SIZE
        )

    # define a model
    EMBED_INP_SIZE = len( tokenizer.word_index ) + 1
    model = CNN_GRU( max_len=MAX_LEN, embed_inp=EMBED_INP_SIZE, embed_size=EMBED_SIZE,
        embedding_matrix=EMBEDDING_MATRIX, embed_trainable=EMBED_TRAINABLE,
        emb_weights_init=EMB_WEIGHTS_INIT, optimizer=OPTIMIZER,
        ker_regularizer=KER_REGULARIZER, drpt=DRPT)

    # save model summaries and model architecture diagrams in the first fold only
    if fold == 1:
        plot_model( model=model, to_file='{}/{}.png'.format( modelsummaries, modelname ), show_shapes=False )

        with open( '{}/{}.txt'.format( modelsummaries, modelname ), 'w' ) as s:
            with redirect_stdout( s ):
                model.summary()

    if isinstance(model.optimizer.learning_rate, tf.Variable):
      K.set_value(model.optimizer.learning_rate, LR_RATE)
    else:
      model.optimizer.learning_rate = LR_RATE
    filepath = modelpath + modelname + str( fold ) + ".keras"

    # train the model with callbacks for early stopping
    f1callback = F1_score_callback(val_data=( xval, yval ), filepath=filepath,patience=PATIENCE, decay=DECAY, decay_rate=DECAY_RATE, decay_after=DECAY_AFTER)
    histmodel = model.fit(xtrain, ytrain, batch_size=BATCH, epochs=NEPOCHS, verbose=0, callbacks=[ f1callback ])

    # save history of all folds
    hist[ 'fold' + str( fold ) ] = histmodel.history.copy()

    # delete trained model object
    del model, histmodel, f1callback
    EMBEDDING_MATRIX = []
    K.clear_session()
    gc.collect()
    # delete pre trained embeddings from the memory
    if ( PRE_TRAINED_FLAG == True ) and ( fold == skf.n_splits ):
        del EMBEDDINGS_INDEX
        gc.collect()

    # load saved model
    loaded_model = load_model( filepath )

    # get predictions (probabilities) for validation and test sets respectively
    valpredictions = loaded_model.predict( xval, verbose=0, batch_size=BATCH )
    testpredictions = loaded_model.predict( xtest, verbose=0, batch_size=BATCH )

    # delete loaded model
    del loaded_model
    K.clear_session()
    gc.collect()

    # optimizer threshold on validation set
    threshold = optimize_threshold( yval, valpredictions )

    # save accuracy, precision, recall, f1 and confusion matrices
    vallabels = ( valpredictions >= threshold ).astype( 'int32' )
    testlabels = ( testpredictions >= threshold ).astype( 'int32' )

    valaccuracy.append( accuracy_score( yval, vallabels ) )
    valprecision.append( precision_score( yval, vallabels ) )
    valrecall.append( recall_score( yval, vallabels ) )
    valf1.append( f1_score( yval, vallabels ) )
    valcm.append( confusion_matrix( yval, vallabels ) )

    testaccuracy.append( accuracy_score( ytest, testlabels ) )
    testprecision.append( precision_score( ytest, testlabels ) )
    testrecall.append( recall_score( ytest, testlabels ) )
    testf1.append( f1_score( ytest, testlabels ) )
    testcm.append( confusion_matrix( ytest, testlabels ) )
    reports.append(classification_report( ytest, testlabels, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))

    print( 'Fold: {:02d} out of {:02d} completed.'.format( fold, skf.get_n_splits() ) )
    print( 'Local System Time: {}'.format( time.strftime( "%I:%M %p", time.localtime() ) ) )

    fold = fold + 1
time_took = time.time() - start_time
print( f"Total runtime: { hms_string( time_took ) }" )
WriteResutls(reports)
